In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from HsvDeger import karart
import csv
import glob


features=[0,0,0,0,0,0,0]

cv_img = []
image_namelist = []
#
for img in glob.glob("./train/benign/*.jpg"):
    image_namelist.append(img)
    n = cv2.imread(img)
    cv_img.append(n)

for img in glob.glob("./train/malignant/*.jpg"):
    image_namelist.append(img)
    n = cv2.imread(img)
    cv_img.append(n)

for s,image_as in enumerate(cv_img):
    print(s)
    print(image_namelist[s])
    img_clone = image_as.copy()
    image = karart(image_as)
    
#---------------Lesion Segmentation  
    lower = np.array([0, 100, 19], dtype = "uint8")
    upper = np.array([17, 200, 120], dtype = "uint8")
    # Finding the colors within the specified limits and applying the mask
    converted = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask  = cv2.inRange(converted, lower, upper)
    
    #--------Finding contours through the mask
    contours, _ = cv2.findContours(mask , cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    areas = [cv2.contourArea(c) for c in contours]
    max_index = np.argmax(areas)
    cnt=contours[max_index]
    stencil = np.zeros(image.shape).astype(image.dtype)
    cv2.fillPoly(stencil,  [cnt] , color=(255,255,255))
    stencil = cv2.cvtColor(stencil, cv2.COLOR_BGR2GRAY)
    cv2.imshow("ff",stencil)
    
    put = cv2.bitwise_and(img_clone, img_clone, mask = stencil )
    #--------Finding colour
    img = put
    height, width, _ = np.shape(img)
    data = np.reshape(img, (height * width, 3))
    data = np.float32(data)
    number_clusters = 2
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    flags = cv2.KMEANS_RANDOM_CENTERS
    _, _, centers = cv2.kmeans(data, number_clusters, None, criteria, 10, flags)
    
    

    rgb_values = []
    for _, color in enumerate(centers):
        rgb = (int(color[2]), int(color[1]), int(color[0]))
        rgb_values.append(rgb)
    for i in rgb_values:
        if(i[0]!=0 and i[1]!=0 and i[2]!=0):
            print("    dominant colour gbr:",i)
            features[2]=i[0]
            features[3]=i[1]
            features[4]=i[2]
    
    #radius
    (x,y),radius = cv2.minEnclosingCircle(cnt)
    print("    Radius:",radius)
    center = (int(x),int(y))
    #pixels to mm has to bee converted
    features[5] = 2*int(radius)
    
    #asymitrycity
       
    width_cutoff = width // 2
    def dice(pred, true, k = 255):
        intersection = np.sum(pred[true==k]) * 2.0
        dice = intersection / (np.sum(pred) + np.sum(true))
        return dice
    
    
    roi=stencil
    radi2=int(radius)
    
    try:
        roi1 = roi[:,:width_cutoff] #left half
        roi2 = roi[:, width_cutoff:] #right half
     
        roi2_mirror = cv2.flip(roi2, 1)
        cv2.imshow('right mirror', roi2_mirror)
        #cv2.imshow('left mirror', roi1)     
        dice_score = dice(roi1, roi2_mirror) 
        
    except IndexError:
        roi1 = roi[:, :(width_cutoff+1)] #left half
        roi2 = roi[:, (width_cutoff):] #right half
        
        roi2_mirror = cv2.flip(roi2, 1)
        cv2.imshow('right mirror', roi2_mirror)
               
        dice_score = dice(roi1, roi2_mirror)     
        print("roi1",roi1.shape)
        print("roi2",roi2.shape)

    features[0]=round(dice_score,3)
    if(s>1436):
        features[6]=1
    print("Dice Similarity: ", dice_score)
    with open('features.csv', mode='a',newline='') as yeni_dosya:
        yeni_yazici = csv.writer(yeni_dosya, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        yeni_yazici.writerow(features)
    print(features)     
    cv2.imshow("images", image)
    cv2.imshow('soutput',put)
    cv2.imshow("images", image)
cv2.waitKey(0)

0
./train/benign\100.jpg
    dominant colour gbr: (116, 67, 65)
    Radius: 56.33346939086914
Dice Similarity:  0.8002575245453082
[0.8, 0, 116, 67, 65, 112, 0]
1
./train/benign\1000.jpg
    dominant colour gbr: (209, 118, 128)
    dominant colour gbr: (72, 16, 21)
    Radius: 157.68492126464844
Dice Similarity:  0.9418018436095935
[0.942, 0, 72, 16, 21, 314, 0]
2
./train/benign\1001.jpg
    dominant colour gbr: (174, 87, 90)
    Radius: 133.40550231933594
Dice Similarity:  0.7889073690641866
[0.789, 0, 174, 87, 90, 266, 0]
3
./train/benign\1002.jpg
    dominant colour gbr: (160, 81, 78)
    Radius: 33.97803497314453
Dice Similarity:  0.4028457092040907
[0.403, 0, 160, 81, 78, 66, 0]
4
./train/benign\1004.jpg
    dominant colour gbr: (209, 107, 109)
    Radius: 121.67384338378906
Dice Similarity:  0.46084512069965644
[0.461, 0, 209, 107, 109, 242, 0]
5
./train/benign\1005.jpg
    dominant colour gbr: (172, 95, 110)
    Radius: 21.526657104492188
Dice Similarity:  0.45353159851301117
[0

    dominant colour gbr: (212, 99, 109)
    Radius: 104.7235107421875
Dice Similarity:  0.5791864338346624
[0.579, 0, 212, 99, 109, 208, 0]
51
./train/benign\1059.jpg
    dominant colour gbr: (173, 59, 64)
    dominant colour gbr: (209, 113, 116)
    Radius: 157.68492126464844
Dice Similarity:  0.9947280854731694
[0.995, 0, 209, 113, 116, 314, 0]
52
./train/benign\106.jpg
    dominant colour gbr: (189, 107, 83)
    Radius: 98.49722290039062
Dice Similarity:  0.7316285677386026
[0.732, 0, 189, 107, 83, 196, 0]
53
./train/benign\1060.jpg
    dominant colour gbr: (132, 57, 49)
    Radius: 9.761981964111328
Dice Similarity:  0.23300970873786409
[0.233, 0, 132, 57, 49, 18, 0]
54
./train/benign\1062.jpg
    dominant colour gbr: (193, 98, 102)
    Radius: 48.11032485961914
Dice Similarity:  0.5136298421807748
[0.514, 0, 193, 98, 102, 96, 0]
55
./train/benign\1063.jpg
    dominant colour gbr: (210, 103, 94)
    Radius: 54.42665481567383
Dice Similarity:  0.5841121495327103
[0.584, 0, 210, 103,

    dominant colour gbr: (138, 90, 72)
    Radius: 39.14423370361328
Dice Similarity:  0.2405002405002405
[0.241, 0, 138, 90, 72, 78, 0]
103
./train/benign\1116.jpg
    dominant colour gbr: (55, 15, 17)
    dominant colour gbr: (208, 119, 123)
    Radius: 157.68492126464844
Dice Similarity:  0.9685298842525848
[0.969, 0, 208, 119, 123, 314, 0]
104
./train/benign\1117.jpg
    dominant colour gbr: (186, 120, 102)
    Radius: 96.10960388183594
Dice Similarity:  0.4264076047216129
[0.426, 0, 186, 120, 102, 192, 0]
105
./train/benign\1119.jpg
    dominant colour gbr: (145, 84, 96)
    Radius: 8.558721542358398
Dice Similarity:  0.0
[0.0, 0, 145, 84, 96, 16, 0]
106
./train/benign\112.jpg
    dominant colour gbr: (209, 113, 116)
    dominant colour gbr: (156, 50, 47)
    Radius: 157.68492126464844
Dice Similarity:  0.9936359438678204
[0.994, 0, 156, 50, 47, 314, 0]
107
./train/benign\1120.jpg
    dominant colour gbr: (148, 95, 83)
    Radius: 23.866031646728516
Dice Similarity:  0.0
[0.0, 0, 

    dominant colour gbr: (227, 125, 118)
    Radius: 117.65213012695312
Dice Similarity:  0.7503895294484263
[0.75, 0, 227, 125, 118, 234, 0]
151
./train/benign\1171.jpg
    dominant colour gbr: (180, 95, 98)
    Radius: 145.85287475585938
Dice Similarity:  0.690056134723336
[0.69, 0, 180, 95, 98, 290, 0]
152
./train/benign\1172.jpg
    dominant colour gbr: (213, 128, 120)
    dominant colour gbr: (145, 48, 48)
    Radius: 157.33334350585938
Dice Similarity:  0.9970998259895594
[0.997, 0, 145, 48, 48, 314, 0]
153
./train/benign\1174.jpg
    dominant colour gbr: (184, 88, 77)
    Radius: 77.22865295410156
Dice Similarity:  0.349524122087299
[0.35, 0, 184, 88, 77, 154, 0]
154
./train/benign\1175.jpg
    dominant colour gbr: (177, 81, 72)
    Radius: 89.93450927734375
Dice Similarity:  0.7988143756947017
[0.799, 0, 177, 81, 72, 178, 0]
155
./train/benign\1176.jpg
    dominant colour gbr: (232, 131, 124)
    Radius: 109.9930648803711
Dice Similarity:  0.860131010867947
[0.86, 0, 232, 131, 

    dominant colour gbr: (200, 107, 123)
    Radius: 138.70303344726562
Dice Similarity:  0.7524588535284962
[0.752, 0, 200, 107, 123, 276, 0]
199
./train/benign\1225.jpg
    dominant colour gbr: (191, 89, 101)
    Radius: 43.85459899902344
Dice Similarity:  0.007895959126799815
[0.008, 0, 191, 89, 101, 86, 0]
200
./train/benign\1227.jpg
    dominant colour gbr: (150, 90, 73)
    Radius: 45.24491500854492
Dice Similarity:  0.3964308476736775
[0.396, 0, 150, 90, 73, 90, 0]
201
./train/benign\1228.jpg
    dominant colour gbr: (229, 133, 143)
    Radius: 136.87921142578125
Dice Similarity:  0.5080035315342981
[0.508, 0, 229, 133, 143, 272, 0]
202
./train/benign\1229.jpg
    dominant colour gbr: (195, 95, 95)
    Radius: 86.07372283935547
Dice Similarity:  0.6972632769959691
[0.697, 0, 195, 95, 95, 172, 0]
203
./train/benign\123.jpg
    dominant colour gbr: (27, 8, 10)
    dominant colour gbr: (176, 95, 95)
    Radius: 156.277099609375
Dice Similarity:  0.6514663859226951
[0.651, 0, 176, 9

...

    Radius: 157.68492126464844
Dice Similarity:  0.9252695775145837
[0.925, 0, 201, 132, 114, 314, 1]
2430
./train/malignant\775.jpg
    dominant colour gbr: (166, 95, 86)
    Radius: 19.235483169555664
Dice Similarity:  0.3682983682983683
[0.368, 0, 166, 95, 86, 38, 1]
2431
./train/malignant\777.jpg
    dominant colour gbr: (95, 68, 57)
    Radius: 8.24631118774414
Dice Similarity:  0.0
[0.0, 0, 95, 68, 57, 16, 1]
2432
./train/malignant\779.jpg
    dominant colour gbr: (189, 109, 109)
    Radius: 7.611433982849121
Dice Similarity:  0.0
[0.0, 0, 189, 109, 109, 14, 1]
2433
./train/malignant\78.jpg
    dominant colour gbr: (158, 134, 98)
    Radius: 9.999999747378752e-05
Dice Similarity:  0.0
[0.0, 0, 158, 134, 98, 0, 1]
2434
./train/malignant\780.jpg
    dominant colour gbr: (188, 120, 106)
    Radius: 157.33175659179688
Dice Similarity:  0.6741923641703378
[0.674, 0, 188, 120, 106, 314, 1]
2435
./train/malignant\781.jpg
    dominant colour gbr: (165, 117, 95)
    Radius: 48.16929626464

Dice Similarity:  0.43049085139423404
[0.43, 0, 198, 125, 112, 244, 1]
2482
./train/malignant\834.jpg
    dominant colour gbr: (184, 109, 102)
    Radius: 104.89249420166016
Dice Similarity:  0.37162837162837165
[0.372, 0, 184, 109, 102, 208, 1]
2483
./train/malignant\835.jpg
    dominant colour gbr: (162, 84, 75)
    Radius: 23.648242950439453
Dice Similarity:  0.8284854563691073
[0.828, 0, 162, 84, 75, 46, 1]
2484
./train/malignant\837.jpg
    dominant colour gbr: (154, 87, 70)
    Radius: 83.34748077392578
Dice Similarity:  0.8437438937015567
[0.844, 0, 154, 87, 70, 166, 1]
2485
./train/malignant\84.jpg
    dominant colour gbr: (183, 125, 102)
    Radius: 30.536962509155273
Dice Similarity:  0.0
[0.0, 0, 183, 125, 102, 60, 1]
2486
./train/malignant\840.jpg
    dominant colour gbr: (63, 35, 38)
    Radius: 68.61679077148438
Dice Similarity:  0.0
[0.0, 0, 63, 35, 38, 136, 1]
2487
./train/malignant\841.jpg
    dominant colour gbr: (140, 84, 74)
    Radius: 68.55892181396484
Dice Simila

    Radius: 70.78321838378906
Dice Similarity:  0.8548916611950098
[0.855, 0, 141, 82, 59, 140, 1]
2534
./train/malignant\896.jpg
    dominant colour gbr: (71, 32, 37)
    Radius: 46.73873519897461
Dice Similarity:  0.0
[0.0, 0, 71, 32, 37, 92, 1]
2535
./train/malignant\897.jpg
    dominant colour gbr: (175, 104, 56)
    Radius: 72.66587829589844
Dice Similarity:  0.8966740576496675
[0.897, 0, 175, 104, 56, 144, 1]
2536
./train/malignant\898.jpg
    dominant colour gbr: (177, 102, 89)
    Radius: 58.331050872802734
Dice Similarity:  0.8319628926839553
[0.832, 0, 177, 102, 89, 116, 1]
2537
./train/malignant\899.jpg
    dominant colour gbr: (173, 97, 79)
    Radius: 100.15155029296875
Dice Similarity:  0.8420271416454622
[0.842, 0, 173, 97, 79, 200, 1]
2538
./train/malignant\9.jpg
    dominant colour gbr: (157, 111, 81)
    Radius: 30.903356552124023
Dice Similarity:  0.7844682048396173
[0.784, 0, 157, 111, 81, 60, 1]
2539
./train/malignant\90.jpg
    dominant colour gbr: (172, 126, 88)


    dominant colour gbr: (171, 108, 97)
    Radius: 93.70751953125
Dice Similarity:  0.0
[0.0, 0, 171, 108, 97, 186, 1]
2584
./train/malignant\950.jpg
    dominant colour gbr: (187, 118, 93)
    Radius: 93.21490478515625
Dice Similarity:  0.6707706693680355
[0.671, 0, 187, 118, 93, 186, 1]
2585
./train/malignant\951.jpg
    dominant colour gbr: (128, 79, 64)
    Radius: 40.2058219909668
Dice Similarity:  0.8353326063249727
[0.835, 0, 128, 79, 64, 80, 1]
2586
./train/malignant\952.jpg
    dominant colour gbr: (211, 86, 114)
    Radius: 25.129791259765625
Dice Similarity:  0.0
[0.0, 0, 211, 86, 114, 50, 1]
2587
./train/malignant\954.jpg
    dominant colour gbr: (196, 109, 100)
    Radius: 104.5751953125
Dice Similarity:  0.7263227987501921
[0.726, 0, 196, 109, 100, 208, 1]
2588
./train/malignant\955.jpg
    dominant colour gbr: (107, 50, 41)
    Radius: 59.50663375854492
Dice Similarity:  0.6582818162438713
[0.658, 0, 107, 50, 41, 118, 1]
2589
./train/malignant\956.jpg
    dominant colou

-1

In [ ]:
import cv2
img_grayscale = cv2.imread('test.jpg',0)
cv2.imshow('graycsale image',img_grayscale)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('grayscale.jpg',img_grayscale)
